# STA 141B Assignment 2

Due __Februrary 9, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10. 

__Exercise 1__

We will use the [lichess](https://lichess.org/api) API to retrieve some information about the current state of chess in the world. In order to answer below questions, make precise and economical requests. You may use:
```
import requests
import json
import pandas

from datetime import datetime
```

In [1]:
import requests
import json
import pandas
from datetime import datetime

__(a)__ What is the real name of the player `muisback`? 

In [15]:
###Access API Token
def read_key(keyfile):
    with open(keyfile) as f:
        return f.readline().strip("\n")
key = read_key("../hw2/lichessAPItoken.txt")

In [17]:
response = requests.get("https://lichess.org/api/user/muisback", params={"profile":True}, headers= {
    'Authorization' : f'Bearer {key}',
})
account_data = response.json()
full_name = f'{account_data["profile"]["firstName"]}, {account_data["profile"]["lastName"]}'
full_name

'Rauf, Mamedov'

__(b, i)__ Get the username of the last player that played a rapid game against user `athena-pallada` in 2023. __(ii)__ In all games against this user, what is the win-to-loss ratio of `athena-pallada`?

In [22]:
response = requests.get("https://lichess.org/api/games/user/athena-pallada", 
    params={'perfType' : "rapid",'max' : 20}, 
    headers= {'Authorization' : f'Bearer {key}'})
for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                # Process the chunk as needed (e.g., save to a file)
                print(chunk.decode('utf-8'), end='')

[Event "GM athena-pallada simul"]
[Site "https://lichess.org/FWHyjmqs"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "Bacio129"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1500"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A40"]
[Termination "Time forfeit"]

1. d4 1-0


[Event "GM athena-pallada simul"]
[Site "https://lichess.org/fEu5PbLv"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "lawrencepl"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1584"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A00"]
[Termination "Normal"]

1. b4 Nf6 1-0


[Event "GM athena-pallada simul"]
[Site "https://lichess.org/0Bo3Zbbd"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "wranglerek"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1665"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "C42"]
[T

In [26]:
response = requests.get("https://lichess.org/api/games/user/athena-pallada", 
    params = {
        'vs' : 'Bacio129'
    },
    headers = {
        'Authorization' : f'Bearer {key}'
    })
for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                # Process the chunk as needed (e.g., save to a file)
                print(chunk.decode('utf-8'), end='')

[Event "GM athena-pallada simul"]
[Site "https://lichess.org/FWHyjmqs"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "Bacio129"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:34:42"]
[WhiteElo "2837"]
[BlackElo "1500"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A40"]
[Termination "Time forfeit"]

1. d4 1-0


[Event "GM athena-pallada simul"]
[Site "https://lichess.org/7BzFYJsB"]
[Date "2023.11.26"]
[White "athena-pallada"]
[Black "Bacio129"]
[Result "1-0"]
[UTCDate "2023.11.26"]
[UTCTime "17:15:14"]
[WhiteElo "2837"]
[BlackElo "1500"]
[WhiteTitle "GM"]
[Variant "Standard"]
[TimeControl "900+0"]
[ECO "A01"]
[Termination "Time forfeit"]

1. b3 d5 2. Bb2 Nc6 3. e3 e5 4. Bb5 Bd6 5. f4 Bd7 6. Nf3 Nge7 7. Bxc6 Nxc6 8. fxe5 Be7 9. O-O O-O 10. Nc3 Bg4 11. d4 Qd7 12. h3 Bh5 13. Ne2 f6 14. Nf4 Bf7 15. Qe2 fxe5 16. Nxe5 Nxe5 17. dxe5 Be6 18. Rad1 Rad8 19. c4 c6 20. Kh2 Bc5 21. cxd5 cxd5 22. Bd4 Bxd4 23. exd4 Qc6 24. Rc1 Qd7 25. Nxe6 Qxe6 26. Rxf8+ Rxf8 27. Rc7 

The user name of the last player athena-pallada has played against is Bacio129. In all the games against this player, which is two games, Athena-Pallada has won both games, so they have a win ratio of 2:0. 

__(c)__ Consider the top ten players in the bullet leaderboard. __(i)__ Which player has the most bullet games overall? __(ii)__ Which player has played the most bullet games relative to account age in days? __(iii)__ Which player has the worst win-to-loss ratio over all formats?

__(d)__ Get all games from user `manwithavan`. Group them by opening and print the ten most popular. 

In [295]:
openings.value_counts().head(2)

Van't Kruijs Opening                     7
Nimzo-Larsen Attack: Modern Variation    7
dtype: int64

__Exercise 2__

As a public organization, the compensations of employees of all institutions of the University of California are freely accessible. These reports cover UC's career faculty and staff employees, as well as part-time, temporary and student employees. See [here](https://ucannualwage.ucop.edu). Internally, the data requested by the search mask is queried using an undocumented API. For this exercise, you may use: 
```
import requests
import pandas

from json import loads
```

_Hint: If you encounter an error when parsing the data, try to use string methods (e.g., `str.replace`) to deal with them._

__(a)__ Get the compensation information of all UC Davis employees that received a gross pay that exceeded 300000 USD per year for the years 2019 to 2020. Sort the resulting table by year and last name, and print the first six entries.

In [231]:
table.head(2)

,year,location,firstname,lastname,title,gross,regular,overtime,other
0,2019,Davis,LEONARD,ABBEDUTO,PROF-HCOMP,374157.0,315450.0,0.0,58707.0
1,2019,Davis,MEHRDAD,ABEDI,PROF OF CLIN-HCOMP,310945.0,173831.0,0.0,137114.0


 __(b)__ Report the mean compensation for each title type: For gross pay, other pay and overtime pay, report the top six titles together with the number of counts. 

In [254]:
grouped_table.head(2)

,gross,regular,overtime,other,count
title,,,,,
VC AND DEAN SOM,960075.0,759375.0,0.0,200700.0,1
COO MED CTR,877439.5,622980.5,0.0,254459.0,2
